In [ ]:
-- create a customer segmentation model of RFM based on the data in the table "dde-srv-made-market-hk-d001.hk_aggregated.hk_owner_asset"
WITH CustomerRFM AS (
  SELECT
    email,
    MAX(purchase_date) AS LastPurchaseDate,
    COUNT(DISTINCT owner_asset_id) AS Frequency,
    SUM(unit_price) AS MonetaryValue
  FROM
    `dde-sbx-kenneth-lam-s001.TEST.owner_asset_w_price`
  GROUP BY
    email
), RFMSegmentation AS (
  SELECT
    CustomerRFM.email,
    CustomerRFM.LastPurchaseDate,
    CustomerRFM.Frequency,
    CustomerRFM.MonetaryValue,
    NTILE(5) OVER (ORDER BY CustomerRFM.LastPurchaseDate DESC) AS RecencyScore,
    NTILE(5) OVER (ORDER BY CustomerRFM.Frequency) AS FrequencyScore,
    NTILE(5) OVER (ORDER BY CustomerRFM.MonetaryValue) AS MonetaryScore
  FROM
    `CustomerRFM`
)
SELECT
  RFMSegmentation.email,
  RFMSegmentation.LastPurchaseDate,
  RFMSegmentation.Frequency,
  RFMSegmentation.MonetaryValue,
  RFMSegmentation.RecencyScore,
  RFMSegmentation.FrequencyScore,
  RFMSegmentation.MonetaryScore,
  CASE
    WHEN RFMSegmentation.RecencyScore = 5
    AND RFMSegmentation.FrequencyScore = 5
    AND RFMSegmentation.MonetaryScore = 5 THEN 'Champions'
    WHEN RFMSegmentation.RecencyScore = 5
    AND RFMSegmentation.FrequencyScore = 4
    AND RFMSegmentation.MonetaryScore = 4 THEN 'Loyal Customers'
    WHEN RFMSegmentation.RecencyScore = 4
    AND RFMSegmentation.FrequencyScore = 5
    AND RFMSegmentation.MonetaryScore = 5 THEN 'Potential Loyalists'
    WHEN RFMSegmentation.RecencyScore = 3
    AND RFMSegmentation.FrequencyScore = 3
    AND RFMSegmentation.MonetaryScore = 3 THEN 'Need Attention'
    WHEN RFMSegmentation.RecencyScore = 2
    AND RFMSegmentation.FrequencyScore = 2
    AND RFMSegmentation.MonetaryScore = 2 THEN 'At Risk'
    ELSE 'Lost Customers'
  END AS CustomerSegment
FROM
  `RFMSegmentation`;